In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNetCV 
from sklearn.metrics import mean_absolute_error,mean_squared_error,root_mean_squared_error,r2_score

---
# Model Evalution without Hyper Parameter tunning

In [3]:
def model_evaluation(models):
    mae = []
    mse = []
    rmse = []
    score = []
    for model in models :
        model.fit(X_train , y_train)
        y_pred = model.predict(X_test)
        mae.append(mean_absolute_error(y_test , y_pred))
        mse.append(mean_squared_error(y_test , y_pred))
        rmse.append(root_mean_squared_error(y_test , y_pred))
        score.append(r2_score(y_test , y_pred))
    model_performance = {
        "Model" : models,
        "mean_absolute_error" : mae,
        "mean_squared_error" : mse,
        "root_mean_squared_error" : rmse,
        "r2_score" : score
    }
    return pd.DataFrame(model_performance)

In [4]:
dataframe = pd.read_csv(r'dataset.csv')
dataframe.head(3)

,Unnamed: 0,Age,BMI,Children,Sex,Smoker,Region,Medical Cost
0,0,58,15.6,2,1,1,1,17907.54
1,1,24,29.8,0,1,1,0,16312.64
2,2,50,29.0,5,1,0,1,6819.21


In [5]:
X = dataframe.drop("Medical Cost" , axis = 1)
y = dataframe["Medical Cost"]

In [6]:
X_train , X_test , y_train , y_test = train_test_split( X , y , test_size = 0.2 , random_state = 42 )

In [7]:
scaler = StandardScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)

array([[ 0.42961263, -1.71072636, -0.56872694, ...,  1.00075028,
        -1.00225254,  1.33325889],
       [-0.11342351, -0.04704423,  0.63054456, ...,  1.00075028,
         0.99775253, -0.44902386],
       [-1.1361185 ,  0.67629583,  1.14057956, ..., -0.99925028,
         0.99775253, -1.34016524],
       ...,
       [ 0.98407747,  0.89329784,  0.12050955, ..., -0.99925028,
        -1.00225254, -0.44902386],
       [-1.35638061,  0.60396182, -1.42338019, ...,  1.00075028,
        -1.00225254,  0.44211752],
       [ 0.66407403,  1.6889719 ,  0.36863469, ..., -0.99925028,
        -1.00225254, -0.44902386]])

In [8]:
models = [ DecisionTreeRegressor() , LinearRegression() , SVR() , ElasticNetCV() ]

In [9]:
performance_result = model_evaluation(models)
performance_result

,Model,mean_absolute_error,mean_squared_error,root_mean_squared_error,r2_score
0,DecisionTreeRegressor(),349.071005,1.850648e+05,430.191551,0.994966
1,LinearRegression(),253.036231,8.502977e+04,291.598647,0.997687
2,SVR(),6072.547516,5.352823e+07,7316.298736,-0.456162
3,ElasticNetCV(),5978.259730,3.597629e+07,5998.023569,0.021315


---
# Model Evaluation with Hyper Parameter Tunning

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
def performance_result_hyperparameter_tunning(models , params):
    best_params = []
    mae = []
    mse = []
    rmse = []
    score = []
    for i in range(len(models)) :
        grid = GridSearchCV(estimator= models[i] , param_grid= params[i] , scoring = 'r2')
        grid.fit(X_train , y_train)
        best_params.append(grid.best_params_)
        y_pred = grid.predict(X_test)
        mae.append(mean_absolute_error(y_test , y_pred))
        mse.append(mean_squared_error(y_test , y_pred))
        rmse.append(root_mean_squared_error(y_test , y_pred))
        score.append(r2_score(y_test , y_pred))
    model_performance = {
        "Model" : models,
        "best_params" : best_params,
        "mean_absolute_error" : mae,
        "mean_squared_error" : mse,
        "root_mean_squared_error" : rmse,
        "r2_score" : score
    }
    return pd.DataFrame(model_performance)

In [13]:
'''
models_hyper = [ DecisionTreeRegressor() , SVR() , ElasticNetCV() ]
params = {
    0 : dict(criterion = [ 'squared_error' , 'friedman_mse' ] ,
     splitter =  [ 'best' ,  'random' ] ),
    1 : dict(kernel = ['linear', 'poly', 'rbf'],
     gamma = ['scale', 'auto']),
    2 : dict(l1_ratio = [.1, .5, .7, .9, .95, 1],
     selection = ['cyclic', 'random'])
} 
'''

"\nmodels_hyper = [ DecisionTreeRegressor() , SVR() , ElasticNetCV() ]\nparams = {\n    0 : dict(criterion = [ 'squared_error' , 'friedman_mse' ] ,\n     splitter =  [ 'best' ,  'random' ] ),\n    1 : dict(kernel = ['linear', 'poly', 'rbf'],\n     gamma = ['scale', 'auto']),\n    2 : dict(l1_ratio = [.1, .5, .7, .9, .95, 1],\n     selection = ['cyclic', 'random'])\n} \n"

In [14]:
#performance_result_hyperparameter_tunning = performance_result_hyperparameter_tunning(models_hyper , params)

In [15]:
#performance_result_hyperparameter_tunning

---
# Model Evaluation of Ensemblers

In [17]:
from sklearn import ensemble

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor

In [19]:
models_ensemble = [ RandomForestRegressor() , AdaBoostRegressor() , 
                    BaggingRegressor() , ExtraTreesRegressor() , 
                    GradientBoostingRegressor() , HistGradientBoostingRegressor()
                  ]

In [20]:
performance_result_ensembler = model_evaluation(models_ensemble)

In [21]:
performance_result_ensembler

,Model,mean_absolute_error,mean_squared_error,root_mean_squared_error,r2_score
0,"(DecisionTreeRegressor(max_features=1.0, rando...",268.107872,100479.487757,316.984996,0.997267
1,"(DecisionTreeRegressor(max_depth=3, random_sta...",291.923477,125603.010319,354.405150,0.996583
2,(DecisionTreeRegressor(random_state=923185866)...,274.960367,108178.026637,328.904282,0.997057
3,"(ExtraTreeRegressor(random_state=1942930675), ...",278.826572,110003.051608,331.667079,0.997008
4,([DecisionTreeRegressor(criterion='friedman_ms...,255.158355,87079.292691,295.092007,0.997631
5,HistGradientBoostingRegressor(),257.476145,89298.752746,298.828969,0.997571


---
# pickling scaler object

In [23]:
import pickle as pkl
#with open('Scaler.pkl' , 'wb') as file:
#    pkl.dump(scaler , file)

---
# pickling GradientBoostingRegressor

In [47]:
model = GradientBoostingRegressor().fit(X_train , y_train)
#with open('model.pkl' , 'wb') as file:
 #   pkl.dump(model , file)